# Basic GAN
Ref: 골빈해커의 3분 딥러닝 텐서플로맛

*책과 버전이 맞지 않아 오류 주의(공부하면서 수정 예정)

In [1]:
#필요한 라이브러리
import tensorflow as tf

'''
#플레이스홀더 설정 <- 2.0으로 업데이트 되면서 사용 불가
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
'''
import matplotlib.pyplot as plt
import numpy as np

#tensorflow 1.X 버전에서는 다음과 같은 방식으로 mnist dataset을 불러온다
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

#tensorflow 2.X 버전
#ref : https://www.tensorflow.org/tutorials/quickstart/beginner?hl=ko
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 2s 0us/step


In [2]:
#하이퍼파라미터 설정

total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128 #생성자의 입력값으로 사용할 노이즈의 크기

신경망 모델 구성

In [5]:
#TensorFlow 1.X 버전
#X = tf.placeholder(tf.float32, [None, n_input])
#Z = tf.placeholder(tf.float32, [None, n_noise])

#어떤 식으로 바꿔야 할지. 고민해볼 것
#X = tf.Variable(tf.ones(shape=[None, n_input]), tf.float32)
#Z = tf.Variable(tf.ones(shape=[None, n_noise]), tf.float32)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [12]:
#생성자 신경망에 사용할 변수 설정
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [13]:
#구분자 신경망에 사용할 변수 설정
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [14]:
#생성자 신경망 구성

def generator(noise_z):
    hidden = tf.nn.relu(
        tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(
        tf.matmul(hidden, G_W2) + G_b2)
    
    return output

In [15]:
#구분자 신경망 구성

def discriminator(inputs):
    hidden = tf.nn.relu(
        tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(
        tf.matmul(hidden, D_W2) + D_b2)
    
    return output

In [16]:
#무작위 노이즈 생성해주는 함수

def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [17]:
'''
#노이즈 Z를 이용해 가짜 이미지를 만들 생성자 G 만듦.
#G가 만든 가짜 이미지와 진짜 이미지 X를 각각 구분자에 넣어
#입력한 이미지가 진짜인지 가짜인지 판별
'''

G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [18]:
#손실값 구하기

loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

loss_G = tf.reduce_mean(tf.log(D_gene))

In [19]:
#손실값을 이용한 학습

D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

#변수 최적화하는 함수 구성
    #최적화에 쓸 수 있는 함수는 minimize 뿐이므로
    #최적화하려는 loss_D와 loss_G에 음수 부호 붙여준다
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#total_batch = int(mnist.train.num_examples / batch_size)

# 임의 삽입 부분(total_batch를 mnist를 불러오지못해 사용할 수 없다)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
#
loss_val_D, loss_val_G = 0, 0

In [ ]:
for epoch in range(total_epoch):
    for i in range(total_batch): #total_batch 변수 어떻게 정의할지
        batch_xs, batch_ys = mnist.train.next_batch(batch_size) #해결 필요
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
        
        print('Epoch:', '%04d' % epoch, 'D loss: {:.4}'.format(loss_val_D), 'G loss: {:4}'.format(loss_val_G))